# 集成聚类中的对抗图嵌入  
  
## 摘要  

**$\color{red}{\text{聚类集成：}}$ 通过图分割把多个基本分区合并为一个一致性分区。**   
  
然而它由两个局限性:  
  
1、忽略了对原始特征的重用
  
2、利用可学习的图表示来获得一致性划分，目前还处于探索阶段。  
  
我们提出了一种新的对抗图自动编码器（AGAE）模型，将集成聚类引入到一个深度图嵌入过程中。  
  
具体地说，采用图卷积网络作为概率编码器，将特征内容和一致性图信息联合集成，用一个简单的内积层作为解码器，利用编码后的潜在变量（即嵌入表示）重构图。  
  
此外，我们发展了一个对抗性正则化器来指导具有自适应分区相关先验的网络训练。在8个真实数据集上的实验表明，在几种最先进的深度嵌入和集合聚类方法中，AGAE是有效的。   
  
## 引言  
  
**集成聚类(EC)以一组基本分区（BPs）和目标为输入，通过将所有的BPs集成到一个一致的BPs中，以提供健壮的聚类结果。**  
  
实现这个一致性目标的一个代表性和有效的方法是**基于协联矩阵（co-association matrix），它本质上是将数据点之间的成对相似性编码到分类数据（即BPs）上，并与图划分问题建立了紧密的联系。**  本文**将由协关联矩阵定义的图称为一致图。** 以前的工作表明一致性图能够通过利用基本分区之间的多样性来捕捉各种簇结构。  
  
然而，尽管基于一致性图的EC方法已经取得了令人满意的性能，但是仍然存在两个局限性。  
  
1、 因为一致性图只是用来分类，它忽略了再使用其丰富的原始特征，这可能严重的造成聚类性能下降。  
  
2、现有的一致性图的划分方法没有去学习可识别的低维图特征。  
  
<br>   
    
## （AGAE）模型所用到的方法  

#### 一致性图  （ Consensus Graph）

给定来自K个真实类别的N个样本，$C=\{c_1,c_2,……，c_k\} \quad X=\{x_1,x_2,……，x_N\}\quad$ 产生了M个基本分区（BPS），$\prod=\{\pi_1,\pi_2,……，\pi_m\}$  
每个基本分区通过运行现有的聚类算法获得。获得标签集合$\prod_r=\{\pi_r(x_1),\pi_r(x_2),……，\pi_r(x_N)\} \quad$(这里一个基本分区：指的是一个聚类算法对所有样本的一个可能划分。)$\quad 1<r<M\quad 1<\pi_r(x_i)<K_r,\quad 1<i<N \quad$(这里$K_r$是第r个基本分区的聚类类别的个数)  
  
* 聚类集成（Ensemble clustering  -EC)  
  
  在基本分区（BPs）的类别信息上，产生关联矩阵（ co-association matrix），在关联矩阵上使用基于图的方法来做分割块。  
  
  <br>
    
* 关联矩阵的定义($S\in R^{N \times N}$)：    
  
  $$S_{ij}=\frac{1}{M}\sum_{r=1}^M \delta(\pi_r(x_i),\pi_r(x_j)) $$  
    
  其中$\delta(\pi_r(x_i),\pi_r(x_j)$表示如果样本$x_i,x_j$在基本分区r中属于同一类别值为1，否则值为0  ；  
    
   $S_{ij}$表示样本$x_i,x_j$在M个基本分区中属于同一个类别的均值。 （矩阵S也可以叫做样本对的亲和力矩阵：**代表了两个样本分配到同一个聚类簇的可能性**）  
     
  <br>  
  
* 可能存在的问题：  
   1、S中可能存在噪声  
   2、由于不同的基本分区的不一致性，导致存在野值。  
     
  使得图划分出现错误。为了缓解这种情况，重建关联矩阵。
  
  $$S=sign([S-\tau]_+)\bigodot S$$   
    
  对$S_{ij}$ 做一个阈值操作；如果$S_{ij}>\tau \quad S_{ij} =S_{ij}\quad$否则$S_{ij}=0$  
  操作$\bigodot $ 表示两个矩阵对应元素相乘。  
  越阶函数$sign([S-\tau]_+)$里$S_{ij}>\tau  $分配为1，否则 为 0  
  正部函数$[𝑥]_+$ 意思是取 𝑥和0 中较大的那个  
    
  **使用这种方法来去除低系数项，保留高系数项，可以建立一个稀疏健壮的关联矩阵图** 来作为AGAE的输入 
    

<br>  


#### 图卷积网络（GCN）  

图卷积网络扩展了卷积神经网络，从低维规则网络到高维图结构。把GCN作为一个基本块
  
   
<br> 

#### Inner product Decoder
  
  从特征还原关联矩阵
   
   $$P(S|Z)=\prod_{i=1}^N \prod_{j=1}^N S(Z_iZ_j^T)$$  
     
<br>  

#### 自适应相关分区的先验  
  
  最近的深层生成模型主要在潜在变量上使用固定的各向同性高斯先验，这缺乏聚类任务的明确指导。因此，我们将我们的配分相关先验公式表示为高斯混合函数。  
    
 **通过使用式（11），提出的AGAE试图将嵌入空间分解成不同的区域。此外，我们的高斯混合模型（GMM）先验值根据分割结果H自适应地更新，这利用等式（12）进一步推动学习一个分离良好的聚类结构（即，更清晰的分布）**     
   
<br> 

#### 对抗性正则化